<a href="https://colab.research.google.com/github/YujiSue/BioInfoTools/blob/master/NGS%E8%A7%A3%E6%9E%90onColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NGS解析用ノート
* このノートは、次世代シーケンス (NGS) のデータ解析をGoogle Colab上で行うためのノートです。  
* 解析用のコンピュータがない、次世代シーケンスのデータ解析がどんなものか試してみたい、といった方が対象になります。  
<font color="red">※Colabで使えるリソースの制限上、大きなサイズのシーケンスデータの解析や、多量のメモリを必要とする解析などは、時間がかかったり途中でエラーが出る可能性があります。</font>  


## はじめに
下のコードセルを実行してください。  
各解析に必要なモジュールの導入などを行います。  

In [6]:
#@title 左にある実行ボタンを押してください
#@markdown * このノートで使用できるソフトとバージョン一覧<br>
#@markdown >[SRAtools](https://github.com/ncbi/sra-tools): v2.11.2<br>
#@markdown [fastp](https://github.com/OpenGene/fastp): v0.23.2<br>
#@markdown [BWA](https://github.com/lh3/bwa): v0.7.17<br>
#@markdown [TVC (TorrentSuite)](https://github.com/iontorrent/TS): v5.12.1<br>
#@markdown [Samtools](http://www.htslib.org/): v1.14<br>
#@markdown [BCFtools](http://www.htslib.org/): v1.14<br>
#@markdown [Picard](https://github.com/broadinstitute/picard): v2.26.9<br>
#@markdown [GATK](https://gatk.broadinstitute.org/hc/en-us): v4.2.4.0<br>
#@markdown [Bowtie2](http://bowtie-bio.sourceforge.net/bowtie2/index.shtml): v2.4.4<br>
#@markdown [STAR](https://github.com/alexdobin/STAR): v2.7.9a<br>
#@markdown [Cufflinks](http://cole-trapnell-lab.github.io/cufflinks/): v2.2.1<br>
#@markdown [MACS2](https://github.com/macs3-project/MACS): v2.2.7.1<br>
#@markdown [MEME Suite](https://meme-suite.org/meme/index.html): v5.4.1<br>

#!pip install -q git+https://github.com/YujiSue/ysngs.git
#import ysngs
#from ysngs import config
#from ysngs import installer
#from ysngs import apprun
!wget https://raw.githubusercontent.com/YujiSue/ysngs/main/ysngs/common.py
!wget https://raw.githubusercontent.com/YujiSue/ysngs/main/ysngs/install.py
!wget https://raw.githubusercontent.com/YujiSue/ysngs/main/ysngs/run.py

import os
import common
import install
import run
cfg = common.config()
cfg.setWorkSpace('/content')
cfg.setDefault()
cfg.makeDirs()
loader = install.installer(cfg)
loader.install('SAM')
app = run.apprun(cfg)

--2022-02-09 05:00:31--  https://raw.githubusercontent.com/YujiSue/ysngs/main/ysngs/common.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2146 (2.1K) [text/plain]
Saving to: ‘common.py.2’

common.py.2         100%[===================>]   2.10K  --.-KB/s    in 0s      

2022-02-09 05:00:31 (40.9 MB/s) - ‘common.py.2’ saved [2146/2146]

--2022-02-09 05:00:32--  https://raw.githubusercontent.com/YujiSue/ysngs/main/ysngs/install.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25789 (25K) [text/plain]
Saving to: ‘install.py.2’



# このノートで解析できること
1. Variant Call (VCF出力)
2. RNA seq (FPKM, TPM出力)
3. ChIP seq (ピークコールとモチーフ検索)

# 1. Variant Call

## 1-1. 解析対象のファイルを準備する
以下の３つの方法のうち、どれか１つ選んで解析対象のファイルを用意してください

### 1-1-1. Samtoolに付随するサンプルを利用する(チュートリアル)
このまま1-2に進んでください。


### 1-1-2. NCBI SRAから解析してみたいファイルをダウンロードする


In [ ]:
#@title ダウンロードしたいIDを指定して実行ボタンを押してください
SRA_ID = 'SRR000001' #@param {type:'string'}
loader.install('SRA')
!vdb-config --interactive
res = runner.downloadSRA(srid=SRA_ID, output='/content/Sample')

Check SRA ... Installed.
2022-01-31T08:17:09 vdb-config.2.11.2 fatal: SIGNAL - Segmentation fault 
Run: > fasterq-dump SRR000001 -O /content/Sample -e 8


### 1-1-3. 自前のデータを利用する
Google DriveやGoogle Cloud Storageにデータがある場合は、該当するファイルをColabのストレージにコピーしてください。  
その他のクラウドストレージにあるファイルはwgetやcurlコマンドでコピーしてください。  
ローカルマシン上のファイルのアップロードは時間がかかるのでお勧めしません。

## 1-2. マッピングとVCFの出力


### パイプライン図


![VC pipeline.png](https://github.com/YujiSue/BioInfoTools/blob/master/image/VC%20pipeline.png?raw=true)

### 実行プログラム

In [ ]:
#@title パラメータをセットして左の実行ボタンを押してください
pref = {}
pref['root'] = '/content'
#@markdown * マッピングの設定
マッピング = True #@param {type: 'boolean'}
pref['mapping'] = マッピング
マッピングソフト = 'BWA' #@param ['BWA', 'Bowtie2']
pref['mapper'] = マッピングソフト
if マッピングソフト == 'BWA':
  loader.install('BWA')
elif マッピングソフト == 'Bowtie2':
  loader.install('Bowtie2')
ターゲットシーケンス = False #@param {type: 'boolean'}
pref['targeted_seq'] = ターゲットシーケンス
ターゲット領域 = '' #@param {type: 'string'}
pref['target'] = ターゲット領域
リードグループ情報付加 = True #@param {type: 'boolean'}
pref['add_rg'] = リードグループ情報付加
リードグループID = 'TEST' #@param {type: 'string'}
pref['read_group_id'] = リードグループID
サンプル名 = 'sample' #@param {type: 'string'}
pref['sample_name'] = サンプル名
プラットフォーム = 'illumina' #@param {type: 'string'}
pref['platform'] = プラットフォーム
#@markdown * リファレンスの情報
リファレンスファイル = 'ex1.fa' #@param {type: 'string'}
リファレンス名 = 'ex1' #@param {type: 'string'}
pref['refdir'] = '/content/Reference',
pref['reference'] = os.path.join('/content/Reference', リファレンスファイル)
pref['refname'] = リファレンス名
#@markdown * バリアント検出
バリアント検出ソフト = 'GATK' #@param ['GATK', 'BCF', 'TVC']
if バリアント検出ソフト == 'GATK':
  loader.install('GATK')
elif バリアント検出ソフト == 'TVC':
  loader.install('TVC')
elif バリアント検出ソフト == 'BCF':
  loader.install('BCF')
パラメータセット = '' #@param {type: 'string'}
モチーフファイル = '' #@param {type: 'string'}
pref['paramdir'] = '/content/Preference'
pref['vcaller'] = バリアント検出ソフト
pref['vcparam'] = パラメータセット
pref['vcmotif'] = モチーフファイル
#@markdown * シーケンスデータの情報
シーケンスデータの形式 = 'fastq' #@param['fastq', 'bam']
シーケンスデータの種類 = 'single' #@param['single', 'paired']
シーケンスファイル１ = '/content/Test/ex1.fq' #@param {type: 'string'}
シーケンスファイル２ = '' #@param {type: 'string'}
pref['iformat'] = シーケンスデータの形式
pref['seqtype'] = シーケンスデータの種類
if pref['seqtype'] == 'single':
  pref['inputs'] = [シーケンスファイル１]
else:
  pref['inputs'] = [シーケンスファイル１,シーケンスファイル２]   
#@markdown * 結果ファイルの情報
出力ファイル名 = 'ex1' #@param {type: 'string'}
pref['outdir'] = '/content/Output'
pref['output_name'] = 出力ファイル名
#@markdown オプション
アダプター除去 = False #@param {type: 'boolean'}
pref['cutadapt'] = アダプター除去
pref['adapter_site'] = 'both'
アダプター配列 = ''#@param {type: 'string'}
pref['adapter_seq'] = アダプター配列
クオリティチェック = False #@param {type: 'boolean'}
pref['fqfliter'] = クオリティチェック
シーケンス長下限値 = 20 #@param {type: 'raw'}
pref['fq_minlen'] = シーケンス長下限値
シーケンスクオリティ下限値 = 15 #@param {type: 'raw'}
pref['fq_qual'] = シーケンスクオリティ下限値
シーケンスの単調性 = 30 #@param {type: 'raw'}    
pref['fq_complex'] = シーケンスの単調性
重複リード検出 = True #@param {type: 'boolean'}
pref['mark_dup'] = 重複リード検出
スプリットリード検出 = True #@param {type: 'boolean'}
pref['detect_sr'] = スプリットリード検出
シーケンスクオリティの再計算 = False #@param {type: 'boolean'}
pref['recal_seq'] = シーケンスクオリティの再計算
pref['use_hotspot'] = False
既知の変異 = '' #@param {type: 'string'}
pref['known_site'] = 既知の変異
バリアントクオリティの再計算 = False #@param {type: 'boolean'}
pref['recal_var'] = バリアントクオリティの再計算
pref['has_control'] = False
pref['control_input'] = ''
スレッド数 =  16 #@param {type: 'raw'}
pref['thread_num'] = スレッド数
使用RAM = 16 #@param {type: 'raw'}
pref['use_ram'] = 使用RAM
pref['gpgpu'] = False  
#コマンドのみ出力 = False #@param {type: 'boolean'}
pref['cmd_only'] = False
pref['verbose'] = True
pref['tmp'] = '',
pref['mediates'] = []
途中産物のファイルを削除 = True #@param {type: 'boolean'}
pref['remove_mediates'] = 途中産物のファイルを削除

%cd /content
from common import config
from run import apprun
import getopt
import os
import sys

def preprocess(pref, app):
  # Cut adapter
  if pref['cutadapt'] and pref['iformat'] == 'fastq':
    for f in pref['inputs']:
      name, ext = os.path.splitext(f)
      ofile = name + '_cut' + ext
      app.runCutter(adaptor = pref['adapter_seq'], site = pref['adapter_site'], input = f, output = ofile)
      pref['mediates'].append(ofile)
      f = ofile
  if pref['fqfliter'] and pref['iformat'] == 'fastq':
    for f in pref['inputs']:
      name, ext = os.path.splitext(f)
      ofile = name + '_filtered' + ext
      app.runFastQFilter(input = f, output = ofile, param = {
        'min_qual': pref['fq_qual'], 'min_len': pref['fq_minlen'], 'min_complex': pref['fq_complex'], 'thread': pref['thread_num']
      })
      pref['mediates'].append(ofile)
      f = ofile

def mapping(pref, app):
  pref['rg_info'] = makeReadGroupInfo(pref)
  
  # BAM => Fastq
  if pref['iformat'] == 'bam':
    app.runSamtool2Fq(seqtype = pref['seqtype'], input = pref['inputs'][0], outdir = app.cfg.OUT_DIR, outname = pref['output_name'] + '_raw')
    if pref['seqtype'] == 'single':
      pref['inputs'][0] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_raw.fq')
    elif pref['seqtype'] == 'paired':
      pref['inputs'][0]= os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_raw_1.fq')
      pref['inputs'].append(os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_raw_2.fq'))
    for f in pref['inputs']:
      pref['mediates'].append(f)
  
  # Fastq => SAM
  pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_mapped.sam')
  if len(pref['refname']) == 0:
      pref['refname'], ext = os.path.splitext(os.path.basename(pref['reference']))
  
  # BWA-MEM
  if pref['mapper'] == 'BWA':
    app.runBWA(seqtype = pref['seqtype'], input = pref['inputs'], ref = pref['refname'], output = pref['output'], \
             option={'thread':pref['thread_num'], 'checksr':pref['detect_sr'], 'refpath':pref['reference'], 'addRG':pref['add_rg'], 'rgroup':pref['rg_info']})
  # Bowtie2
  elif pref['mapper'] == 'Bowtie2':
    app.runBowtie2(seqtype = pref['seqtype'], input = pref['inputs'], ref = pref['refname'], output = pref['output'], \
             option={'thread':pref['thread_num'], 'checksr':pref['detect_sr'], 'refpath':pref['reference'], 'addRG':pref['add_rg'], 'rgroup':pref['rg_info']})
  # STAR
  
  pref['mediates'].append(pref['output'])
  pref['input'] = pref['output']
  os.chdir(app.cfg.WORK_SPACE)

  # SAM -> BAM
  pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_mapped.bam')
  app.runSamtool2BAM(input = pref['input'], output = pref['output'], option={
    'thread':pref['thread_num']
    })
  pref['mediates'].append(pref['output'])

  # Sort BAM
  pref['input'] = pref['output']
  pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_sorted.bam')
  app.runSamtoolSort(input = pref['input'], output = pref['output'], option={
    'thread':pref['thread_num'], 'ram': pref['use_ram']/4
    })
  pref['mediates'].append(pref['output'])
  pref['input'] = pref['output']

  # Mark duplicate
  if pref['mark_dup']:
    pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_marked.bam')
    pref['metric'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_metric.txt')
    app.runPicardMD(input = pref['input'], output = pref['output'], metric = pref['metric'])
    pref['mediates'].append(pref['output'])
    pref['input'] = pref['output']

  # Recalibration sequence quality
  if pref['recal_seq']:
    pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '.bam')
    app.runGATKBRecal(input = pref['input'], output = pref['output'], ref = pref['reference'], known = pref['known_site'], option = {'ram': pref['use_ram'] })
  else:
    pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '.bam')
    os.system('mv ' + pref['input'] + ' ' + pref['output'])
  pref['input'] = pref['output']

  # Make index
  app.runSamtoolIndex(input = pref['input'])
  
def cleanup(pref):
  for f in pref['mediates']:
    os.system('rm ' + f)

def varcall(pref, app):
  # Preprocessing fastq
  preprocess(pref, app)
  
  # Mapping fastq
  if pref['mapping']:
    mapping(pref, app)
  else:
    pref['input'] = pref['inputs'][0]
  
  # Variant call
  if pref['recal_var']:
    pref['output']  = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_raw')
  else:
    pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'])
  # GATK HaplotypeCaller
  if pref['vcaller'] == 'GATK':
    app.runGATKVarCall(input = pref['input'], output = pref['output'], ref = pref['reference'], option = {'ram': pref['use_ram']})
    pref['input'] = pref['output'] + '.vcf.gz'
  # BCFtools mpileup and call
  elif pref['vcaller'] == 'BCF':
    app.runBCFVarCall(input = pref['input'], output = pref['output'], ref = pref['reference'])
    pref['input'] = pref['output'] + '.vcf.gz'
  # TorrentSuite variant caller
  elif pref['vcaller'] == 'TVC':
    tvcopt = {
      'thread': pref['thread_num'],
      'param' : os.path.join(app.cfg.PREFERENCE_DIR, pref['vcparam']), 
      'motif' : os.path.join(app.cfg.PREFERENCE_DIR, pref['vcmotif']) 
    }
    if pref['targeted_seq']:
      tvcopt['target'] = pref['target']
    if pref['use_hotspot']:
      tvcopt['hotspot'] = pref['known_site']
    if pref['has_control']:
      tvcopt['control'] = pref['control_input']
    app.runTVC(input = pref['input'], output = cfg.OUT_DIR, ref = pref['reference'], option = tvcopt)
  # google deep variant
  elif pref['vcaller'] == 'GDV':
    gdvopt = {
      'gpu': pref['gpgpu'],
      'processor': pref['thread_num']
      }
    if pref['targeted_seq']:
      gdvopt['target'] = pref['target']
    app.runGDVCall(input = pref['input'], output = pref['output'], ref = pref['reference'], option = gdvopt)
  
  # Recalibration variant quality
  if pref['recal_var']:
    pref['mediates'].append(pref['output'] + '.vcf.gz')
    pref['mediates'].append(pref['output'] + '.vcf.gz.tbi')
    pref['output'] = os.path.join(cfg.OUT_DIR, pref['output_name'])
    app.runGATKVRecal(input = pref['input'], output = pref['output'], ref = pref['reference'], option = {'ram': pref['use_ram']})
  
  # Erase intermediate files
  if pref['remove_mediates']:
    cleanup(pref)

def makeReadGroupInfo(pref):
  return { 
    'ID': pref['read_group_id'], 
    'SM': pref['sample_name'],
    'PL': pref['platform']
  }

varcall(pref, app)

Check BWA ... Not installed.
Install BWA  ...
  Download source codes ...
  Completed.
  Compile completed.
Installed.
>  Program: bwa (alignment via Burrows-Wheeler transformation)
>  Version: 0.7.17-r1188
Check GATK ... Not installed.
Install GATK ...
  Downloads sources ...
  Completed.
Completed.
>  Using GATK jar /content/MyApp/gatk/gatk-package-4.2.4.0-local.jar
/content
Run: > bwa index -p /content/Reference/ex1 /content/Reference/ex1.fa
 > Completed.

Run: > bwa mem -t 16 -M -R "@RG\tID:TEST\tSM:sample\tPL:illumina" ex1 /content/Test/ex1.fq > /content/Output/ex1_mapped.sam
 > Completed.

Run: > samtools view -@ 16 -b -o /content/Output/ex1_mapped.bam /content/Output/ex1_mapped.sam
 > Completed.

Run: > samtools sort -l1 -T tmp -@ 16 -m 4000.0M -O bam -o /content/Output/ex1_sorted.bam /content/Output/ex1_mapped.bam
Error:
 > [bam_sort] -m setting (4000 bytes) is less than the minimum required (1M).
 > 
 > Trying to run with -m too small can lead to the creation of a very large n

## RNA-seq

## ChIP-seq